In [182]:
import numpy as np
import pandas as pd
import math
from scipy.stats import pearsonr

In [183]:
def rcm_result_protol(some_anime_id,some_result,df):
    alist = []
    for i in range(len(some_anime_id)):
        for j in range(len(some_result)):
            similarity = anime_Jaccard(df,some_anime_id[i],some_result[j])
            alist.append(
                {
                    "rcm_anime_id":some_result[j],
                    "protol_anime_id":some_anime_id[i],
                    "weight":similarity
                }
            )
    unique_dict = {}
    sorted_list = sorted(alist, key=lambda x: x['weight'], reverse=True)
    for item in sorted_list:
        rcm_anime_id = item['rcm_anime_id']
        if rcm_anime_id not in unique_dict:
            unique_dict[rcm_anime_id] = item

    unique_list = list(unique_dict.values())

    return unique_list

In [184]:
def rcm_result_with_slopeone(some_protol_result,user_id,df):
    list = []
    for i in range(len(some_protol_result)):
        list.append({
            "rcm_anime_id":some_protol_result[i]["rcm_anime_id"],
            "protol_anime_id":some_protol_result[i]["protol_anime_id"],
            "predict_rating":df.loc[(df['user_id'] == user_id) & (df['anime_id'] == some_protol_result[i]["protol_anime_id"])]['rating'].tolist()[0]+anime_rating_deficiency(df,some_protol_result[i]['protol_anime_id'],some_protol_result[i]['rcm_anime_id'])
        })
    return sorted(list,key=lambda x: x['predict_rating'], reverse=True)
    

In [185]:
def find_similiar_user(vector,df):
    list=[]
    matrix = []
    for i in range(1000):
        vector_or = df.loc[i+1].fillna(0)
        # print(i)
        pc = pearsonr(vector,vector_or)
        list.append({
            "user_id":i+1,
            "statistic":pc[0],
            "pvalue":pc[1]
        })
    sorted_list = sorted(list, key=lambda x: x['statistic'], reverse=True)
    final_list = []
    for i in range(100):
        final_list.append(sorted_list[i])
        matrix.append(df.iloc[sorted_list[i]["user_id"]].fillna(0))
    # return final_list
    return pd.DataFrame(matrix).mean().sort_values(ascending=False).head(100).index.tolist()

In [186]:
def remove_dumplicate_element(alist,blist):
    result = [x for x in alist if x not in blist]
    return result

In [187]:
def standby_anime(some_list,df):
    alist=[]
    for i in range(20):
        filtered_df = df[df['user_id'] == some_list[i]["user_id"]]
        rated_anime_ids = filtered_df['anime_id'].tolist()
        alist+=rated_anime_ids
    return list(set(alist))

In [188]:
def anime_Jaccard(df,anime_x,anime_y):
    total_count = df[(df['anime_id'] == anime_x) | (df['anime_id'] == anime_y)]['user_id'].nunique()
    anime1_ratings = df[df['anime_id'] == anime_x]
    anime2_ratings = df[df['anime_id'] == anime_y]
    merged_ratings = pd.merge(anime1_ratings, anime2_ratings, on='user_id')
    return merged_ratings.shape[0]/math.sqrt(total_count)

In [189]:
def anime_rating_deficiency(df,anime_x,anime_y):
    book1_ratings = df[df['anime_id'] == anime_x]
    book2_ratings = df[df['anime_id'] == anime_y]
    merged_ratings = pd.merge(book1_ratings, book2_ratings, on='user_id')
    return merged_ratings.mean()["rating_x"]-merged_ratings.mean()["rating_y"]

In [190]:
anime = pd.read_csv("./anime_data/anime.csv")
rating = pd.read_csv("./anime_data/rating.csv")

In [191]:
rating = rating.drop_duplicates(subset=['user_id', 'anime_id'])
rating['rating'] = rating['rating'].clip(lower=0)
user_rating_matrix = rating.pivot(index="user_id",columns="anime_id",values="rating")

In [200]:
user_rating_matrix.columns

Index([    1,     5,     6,     7,     8,    15,    16,    17,    18,    19,
       ...
       34283, 34324, 34325, 34349, 34358, 34367, 34412, 34475, 34476, 34519],
      dtype='int64', name='anime_id', length=11200)

In [202]:
vec = np.zeros([1,11200])

In [204]:
pd.DataFrame(vec,columns=user_rating_matrix.columns)

anime_id,1,5,6,7,8,15,16,17,18,19,...,34283,34324,34325,34349,34358,34367,34412,34475,34476,34519
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
user_id = 3

In [193]:
a=user_rating_matrix.loc[user_id].fillna(0)

some_list = find_similiar_user(a,user_rating_matrix)
rated_anime_ids = rating[rating['user_id'] == user_id]['anime_id'].tolist()
some_list = remove_dumplicate_element(some_list,rated_anime_ids)[:20]

C:\Users\19445\AppData\Local\Temp\ipykernel_27328\4123079746.py:7: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pc = pearsonr(vector,vector_or)


In [194]:
some_list

[6547,
 10620,
 2167,
 1575,
 9756,
 9253,
 2904,
 4181,
 121,
 6746,
 226,
 4224,
 853,
 10793,
 30,
 8525,
 5680,
 10087,
 431,
 4898]

In [195]:
rated_anime_ids

[20,
 154,
 170,
 199,
 225,
 341,
 430,
 527,
 552,
 813,
 1119,
 1121,
 1122,
 1132,
 1292,
 1313,
 1526,
 1535,
 1564,
 1689,
 1764,
 1943,
 2201,
 2404,
 2847,
 3588,
 4026,
 5114,
 5231,
 6178,
 6702,
 6880,
 7695,
 8074,
 9107,
 9135,
 9760,
 9917,
 9919,
 9989,
 10408,
 10507,
 11111,
 11703,
 11737,
 11757,
 11759,
 11771,
 12671,
 14075,
 14093,
 14345,
 14513,
 16498,
 16512,
 16782,
 16894,
 16918,
 17265,
 18097,
 18115,
 18393,
 19315,
 19815,
 20021,
 20159,
 20507,
 20583,
 21507,
 21881,
 22199,
 22297,
 22319,
 22547,
 22729,
 23301,
 23321,
 23333,
 23755,
 24415,
 26243,
 27631,
 27899,
 28121,
 28171,
 28223,
 28497,
 28701,
 28891,
 29854,
 30276,
 30503,
 31043,
 31859]

In [196]:
rated_anime_ids = [x for x in rated_anime_ids if x != 11757]

In [197]:
rated_anime_ids

[20,
 154,
 170,
 199,
 225,
 341,
 430,
 527,
 552,
 813,
 1119,
 1121,
 1122,
 1132,
 1292,
 1313,
 1526,
 1535,
 1564,
 1689,
 1764,
 1943,
 2201,
 2404,
 2847,
 3588,
 4026,
 5114,
 5231,
 6178,
 6702,
 6880,
 7695,
 8074,
 9107,
 9135,
 9760,
 9917,
 9919,
 9989,
 10408,
 10507,
 11111,
 11703,
 11737,
 11759,
 11771,
 12671,
 14075,
 14093,
 14345,
 14513,
 16498,
 16512,
 16782,
 16894,
 16918,
 17265,
 18097,
 18115,
 18393,
 19315,
 19815,
 20021,
 20159,
 20507,
 20583,
 21507,
 21881,
 22199,
 22297,
 22319,
 22547,
 22729,
 23301,
 23321,
 23333,
 23755,
 24415,
 26243,
 27631,
 27899,
 28121,
 28171,
 28223,
 28497,
 28701,
 28891,
 29854,
 30276,
 30503,
 31043,
 31859]

In [198]:
protol_result=rcm_result_protol(rated_anime_ids,some_list,rating)
rcm_result_with_slopeone(protol_result,user_id,rating)

[{'rcm_anime_id': 5680,
  'protol_anime_id': 1535,
  'predict_rating': 10.895958083832335},
 {'rcm_anime_id': 226,
  'protol_anime_id': 1535,
  'predict_rating': 10.8406017994985},
 {'rcm_anime_id': 4898,
  'protol_anime_id': 1535,
  'predict_rating': 10.753599306157849},
 {'rcm_anime_id': 10793,
  'protol_anime_id': 16498,
  'predict_rating': 10.68702073274638},
 {'rcm_anime_id': 30,
  'protol_anime_id': 1535,
  'predict_rating': 10.438846062147341},
 {'rcm_anime_id': 10620,
  'protol_anime_id': 16498,
  'predict_rating': 10.413223667431778},
 {'rcm_anime_id': 121,
  'protol_anime_id': 1535,
  'predict_rating': 10.409215428446197},
 {'rcm_anime_id': 2167,
  'protol_anime_id': 1535,
  'predict_rating': 10.388179128563745},
 {'rcm_anime_id': 853,
  'protol_anime_id': 1535,
  'predict_rating': 10.38017858579358},
 {'rcm_anime_id': 6746,
  'protol_anime_id': 1535,
  'predict_rating': 10.341810344827586},
 {'rcm_anime_id': 4224,
  'protol_anime_id': 1535,
  'predict_rating': 10.26673685514